In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [2]:
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
# df.to_csv('publication.csv')
# st.write(df)

2026-02-11 01:38:39.813 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:38:39.952 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2026-02-11 01:38:40.128 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-02-11 01:38:40.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:38:40.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
def classify_publication(publication):  

    NAMES = []
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n) 

    for t in publication.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: publication.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: publication.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)

    name_paper = defaultdict(list)
    for name in NAMES:
        papers = []
        for i,(role,TCI,Q,conf_int,conf_nat) in enumerate(zip(publication[name],
                                                            publication['วารสารไทย'],
                                                            publication['วารสารนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับชาติ'])):
            if role in ['First Author', 'Corresponding Author']:
                papers+=[TCI+Q+conf_int+conf_nat]
        name_paper[name.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [','').replace(']','')] = papers

    categories = ['Q1','Q2','Q3','Q4','TCI1','TCI2','อื่นๆ']
    blanktable = pd.DataFrame(np.zeros((len(name_paper.keys()), len(categories)),dtype=int), columns=categories)
    blanktable['name']=name_paper.keys()


    for idx, (name, papers) in enumerate(name_paper.items()):
        for paper in papers:
            for cat in categories:
                if cat in paper:
                    blanktable.loc[idx, cat] += 1
    
    # Reorder columns with 'name' first
    blanktable = blanktable[['name'] + [col for col in blanktable.columns if col != 'name']]
    
    # blanktable.to_csv('publication_classified.csv')

    return blanktable

In [35]:
classify_publication(publication)

,name,Q1,Q2,Q3,Q4,TCI1,TCI2,อื่นๆ
0,นพ.สมเกียรติ ลีละศิธร,0,0,0,0,0,0,0
1,พญ.วัชรา ริ้วไพบูลย์,0,0,0,0,0,0,0
2,ศ.ดร.ทวี เชื้อสุวรรณทวี,1,1,1,0,1,0,0
3,รศ.ดร.อาดัม นีละไพจิตร,0,0,0,0,1,0,3
4,รศ.ดร.ณัฏฐนียา โตรักษา,0,0,0,0,0,0,0
5,ผศ.ดร.อารี ภาวสุทธิไพศิฐ,0,0,0,0,0,0,0
6,ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล,0,0,0,0,0,0,0
7,ผศ.ดร.เจนจิรา เจนจิตรวาณิช,0,0,0,0,0,0,0
8,ดร.รติรส จันทรสมดี,0,0,0,0,1,0,0
9,ดร.อิศวรา ศิริรุ่งเรือง,0,0,0,0,0,0,0


In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

2025-12-24 13:56:05.508 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.340 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-24 13:56:06.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.345 Thre

In [2]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [3]:
st.write(A)

2025-12-17 17:05:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.468 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-17 17:05:31.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
